# Real Dataset Experiment (German Credit Dataset-- Group Fairness)
Generate the dataset for LTR setting


In [9]:
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import colors, ticker, cm

from pprint import pprint
from progressbar import progressbar

%matplotlib inline

In [10]:
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 75

plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = r"\usepackage{subdepth}, \usepackage{type1cm}"

In [11]:
#cd 

In [12]:
from train_yahoo_dataset import on_policy_training
from YahooDataReader import YahooDataReader
import torch
from models import NNModel, LinearModel
from evaluation import evaluate_model

In [13]:
# read data 
import pickle as pkl
dr = YahooDataReader(None)
dr.data = pkl.load(open("GermanCredit/german_train_rank_3.pkl", "rb"))
# shape: (train, test)
vdr = YahooDataReader(None)
vdr.data = pkl.load(open("GermanCredit/german_test_rank_3.pkl","rb"))

In [22]:
print((dr.data[0][0]))

[[-1.36730885 -0.03021913 -0.48976238  1.          0.          0.
   0.          1.          0.          0.          0.          1.
   0.          0.          0.          0.          1.          1.
   0.          0.          0.          0.          1.          0.
   0.          0.          0.          0.          0.        ]
 [ 1.00740741 -0.67140719  0.25695309  1.          0.          0.
   0.          1.          0.          1.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          1.          0.          0.          0.
   0.          0.          1.          0.          0.        ]
 [-0.3118794  -0.10394335 -1.23647786  0.          1.          0.
   0.          1.          0.          0.          1.          0.
   0.          0.          0.          1.          0.          1.
   0.          0.          0.          0.          0.          0.
   0.          1.          0.          0.          0.        ]
 [ 3.47007612 -0.67

In [16]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = Namespace(conditional_model=True, gpu_id=None, progressbar=True, evaluate_interval=250, input_dim=29, 
                 eval_rank_limit=1000,
                fairness_version="asym_disparity", entropy_regularizer=0.0, save_checkpoints=False, num_cores=1,
                pooling='concat_avg', dropout=0.0, hidden_layer=8, summary_writing=False, 
                 group_fairness_version="asym_disparity",early_stopping=False, lr_scheduler=False, 
                 validation_deterministic=False, evalk=1000, reward_type="ndcg", baseline_type="value", 
                 use_baseline=True, entreg_decay=0.0, skip_zero_relevance=True, eval_temperature=1.0, optimizer="Adam",
                clamp=False)
torch.set_num_threads(args.num_cores)
args.progressbar = False 

# protected group??
args.group_feat_id = 3

In [17]:
model_params_list = []
lambdas_list = [0.0, 0.1, 1.0, 10.0, 12.0, 15.0, 20.0, 25.0, 50.0, 100.0]
plt_data_pl = np.zeros((len(lambdas_list)+1, 2))
for i, lgroup in enumerate(lambdas_list):
        torch.set_num_threads(args.num_cores)
        args.lambda_reward = 1.0
        args.lambda_ind_fairness = 0.0
        args.lambda_group_fairness = lgroup
        
        args.lr = 0.001
        args.epochs = 10
        args.progressbar = False
        args.weight_decay = 0.0
        args.sample_size = 25
        args.optimizer = "Adam"

        model = LinearModel(D=args.input_dim)

        model = on_policy_training(dr, vdr, model, args=args)
        print(type(model))
        print(model)
        if i == 0:
            results = evaluate_model(model, vdr, fairness_evaluation=False, group_fairness_evaluation=True, 
                                 deterministic=True, args=args, num_sample_per_query=20)
            print(results)
            plt_data_pl[0] = [results["ndcg"], results["avg_group_asym_disparity"]]
        results = evaluate_model(model, vdr, fairness_evaluation=False, group_fairness_evaluation=True, 
                                 deterministic=False, args=args, num_sample_per_query=20)
        print(results)
        model_params_list.append(model.w.weight.data.tolist()[0])
        print("Learnt model for lambda={} has model weights as {}".format(lgroup, model_params_list[-1]))
        plt_data_pl[i+1] = [results["ndcg"], results["avg_group_asym_disparity"]]

Starting training with the following config
Learning rate 0.001, Weight decay 0.0, Sample size 25
Lambda_reward: 1.0, lambda_ind_fairness:0.0, lambda_group_fairness:0.0
Training....
Evaluating on validation set: iteration 0/500 of epoch 0
Epoch 0, Average Validation: NDCG: 0.8259413256603544, DCG 4.875259149384042, Average Rank 12.022325581395348, ERR 0.11779879003297276

Averages of last 1000 rewards: 0.8417893436580538, ndcgs: 0.8417893436580538, dcgs: 4.995361531669446
Evaluating on validation set: iteration 250/500 of epoch 0
Epoch 0, Average Validation: NDCG: 0.8513427964684684, DCG 5.018649271073693, Average Rank 11.470232558139536, ERR 0.12654318404573606
Training....
Evaluating on validation set: iteration 0/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.8659104750176908, DCG 5.1028950552482755, Average Rank 11.08485049833887, ERR 0.13126812531072574

Averages of last 1000 rewards: 0.8786542057553284, ndcgs: 0.8786542057553284, dcgs: 5.238156846169105
Evaluating on validat


Averages of last 1000 rewards: 0.8928502609821103, ndcgs: 0.8928502609821103, dcgs: 5.279687529317856
Evaluating on validation set: iteration 250/500 of epoch 2
Epoch 2, Average Validation: NDCG: 0.888673324885617, DCG 5.233776900604676, Average Rank 10.41514950166113, ERR 0.13793889418628358
Average Validation Group Exposure disparity: 0.06186917893326223, Group Asymmetric disparity: 0.006361291299259532
Training....
Evaluating on validation set: iteration 0/500 of epoch 3
Epoch 3, Average Validation: NDCG: 0.8950640138321859, DCG 5.268540909342425, Average Rank 10.335747508305648, ERR 0.1403595863084818
Average Validation Group Exposure disparity: 0.05769234892278803, Group Asymmetric disparity: 0.004280608581769506

Averages of last 1000 rewards: 0.9086506437843407, ndcgs: 0.9086506437843407, dcgs: 5.407893870541985
Evaluating on validation set: iteration 250/500 of epoch 3
Epoch 3, Average Validation: NDCG: 0.9007982574530664, DCG 5.301930240894206, Average Rank 10.236079734219269

Training....
Evaluating on validation set: iteration 0/500 of epoch 2
Epoch 2, Average Validation: NDCG: 0.8734055481667524, DCG 5.1445205211580936, Average Rank 10.816212624584718, ERR 0.13321774223827662
Average Validation Group Exposure disparity: 0.05980845434442576, Group Asymmetric disparity: 0.0024169315443842133

Averages of last 1000 rewards: 0.8824876664220894, ndcgs: 0.8824876664220894, dcgs: 5.273864320056618
Evaluating on validation set: iteration 250/500 of epoch 2
Epoch 2, Average Validation: NDCG: 0.8790251986949877, DCG 5.176173260003453, Average Rank 10.688239202657808, ERR 0.1350006168155396
Average Validation Group Exposure disparity: 0.06052533849410457, Group Asymmetric disparity: 0.0018016637415644405
Training....
Evaluating on validation set: iteration 0/500 of epoch 3
Epoch 3, Average Validation: NDCG: 0.8818091733234157, DCG 5.192141495601966, Average Rank 10.579003322259137, ERR 0.13569595127398393
Average Validation Group Exposure disparity: 0.05919735585080


Averages of last 1000 rewards: 0.8376062679577481, ndcgs: 0.8376062679577481, dcgs: 5.011654334536934
Evaluating on validation set: iteration 250/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.8330837940571761, DCG 4.916282431366957, Average Rank 11.875149501661129, ERR 0.12026398754211026
Average Validation Group Exposure disparity: 0.06848229657581918, Group Asymmetric disparity: 0.002195062097522595
Training....
Evaluating on validation set: iteration 0/500 of epoch 2
Epoch 2, Average Validation: NDCG: 0.8367125835360103, DCG 4.933251087275534, Average Rank 11.842458471760798, ERR 0.1215533205222075
Average Validation Group Exposure disparity: 0.0668016380559895, Group Asymmetric disparity: 0.003816188549707884

Averages of last 1000 rewards: 0.8395407874590276, ndcgs: 0.8395407874590276, dcgs: 4.998640114295835
Evaluating on validation set: iteration 250/500 of epoch 2
Epoch 2, Average Validation: NDCG: 0.8356443348930646, DCG 4.928725102155497, Average Rank 11.73946843853820

Training....
Evaluating on validation set: iteration 0/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.8353782233435738, DCG 4.927639266743389, Average Rank 11.909501661129568, ERR 0.12139636544068547
Average Validation Group Exposure disparity: 0.07025451327389104, Group Asymmetric disparity: 0.0018842885720070218

Averages of last 1000 rewards: 0.8404585502066921, ndcgs: 0.8404585502066921, dcgs: 5.051264342283546
Evaluating on validation set: iteration 250/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.8372658169614681, DCG 4.93832618287998, Average Rank 11.864318936877076, ERR 0.12199276496381195
Average Validation Group Exposure disparity: 0.0642242693297326, Group Asymmetric disparity: 0.0012559483832783958
Training....
Evaluating on validation set: iteration 0/500 of epoch 2
Epoch 2, Average Validation: NDCG: 0.8364128391187247, DCG 4.932667561541539, Average Rank 11.81468438538206, ERR 0.12126282409823663
Average Validation Group Exposure disparity: 0.07046867565364773


Averages of last 1000 rewards: 0.8367346138811763, ndcgs: 0.8367346138811763, dcgs: 4.984922308543661
Evaluating on validation set: iteration 250/500 of epoch 0
Epoch 0, Average Validation: NDCG: 0.8279962473292377, DCG 4.887339592384978, Average Rank 12.028637873754153, ERR 0.11876625827178179
Average Validation Group Exposure disparity: 0.07339380656877872, Group Asymmetric disparity: 0.0016113606527295937
Training....
Evaluating on validation set: iteration 0/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.827295921966666, DCG 4.879237509513046, Average Rank 12.042458471760797, ERR 0.11827786372066301
Average Validation Group Exposure disparity: 0.0757378135543056, Group Asymmetric disparity: 9.604779541359298e-05

Averages of last 1000 rewards: 0.8329004902273044, ndcgs: 0.8329004902273044, dcgs: 4.944793540734198
Evaluating on validation set: iteration 250/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.8282624286914616, DCG 4.88619608942421, Average Rank 12.0156146179402

Epoch 0, Average Validation: NDCG: 0.8287582727816367, DCG 4.890188124273045, Average Rank 11.984318936877077, ERR 0.11872653333074952
Average Validation Group Exposure disparity: 0.0706457938243113, Group Asymmetric disparity: 0.0008964290873982722

Averages of last 1000 rewards: 0.8403356843734596, ndcgs: 0.8403356843734596, dcgs: 5.056140154342475
Evaluating on validation set: iteration 250/500 of epoch 0
Epoch 0, Average Validation: NDCG: 0.8308825492391645, DCG 4.903779950008616, Average Rank 11.99966777408638, ERR 0.11985978441841968
Average Validation Group Exposure disparity: 0.06921565001123145, Group Asymmetric disparity: 0.0008797494705923875
Training....
Evaluating on validation set: iteration 0/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.832675176528041, DCG 4.91113279182819, Average Rank 11.937940199335548, ERR 0.12018079433770522
Average Validation Group Exposure disparity: 0.06915608940246959, Group Asymmetric disparity: 9.951668682224024e-05

Averages of last 1

Epoch 0, Average Validation: NDCG: 0.8286148048303926, DCG 4.891592207041928, Average Rank 11.964584717607973, ERR 0.11868479301479036
Average Validation Group Exposure disparity: 0.07071396953140513, Group Asymmetric disparity: 0.0014236241280685246

Averages of last 1000 rewards: 0.8389041070282213, ndcgs: 0.8389041070282213, dcgs: 5.036152824153158
Evaluating on validation set: iteration 250/500 of epoch 0
Epoch 0, Average Validation: NDCG: 0.8315664507398699, DCG 4.90826113454716, Average Rank 11.93076411960133, ERR 0.11986199418204618
Average Validation Group Exposure disparity: 0.07117776532153414, Group Asymmetric disparity: 0.003874580708017564
Training....
Evaluating on validation set: iteration 0/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.8269514447776102, DCG 4.879696228171998, Average Rank 12.076146179401993, ERR 0.11839789138578308
Average Validation Group Exposure disparity: 0.07069075136829187, Group Asymmetric disparity: 0.0020971539659709925

Averages of last 

Epoch 0, Average Validation: NDCG: 0.8254962758682098, DCG 4.8713323865979286, Average Rank 12.005647840531562, ERR 0.11729541817783762
Average Validation Group Exposure disparity: 0.07351021318301143, Group Asymmetric disparity: 0.0015827709025595682

Averages of last 1000 rewards: 0.8349219657245179, ndcgs: 0.8349219657245179, dcgs: 4.969833899290672
Evaluating on validation set: iteration 250/500 of epoch 0
Epoch 0, Average Validation: NDCG: 0.8385243143005944, DCG 4.944071994180271, Average Rank 11.880863787375416, ERR 0.12254073198302466
Average Validation Group Exposure disparity: 0.06645688935262829, Group Asymmetric disparity: 0.0011248544365946211
Training....
Evaluating on validation set: iteration 0/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.8276177125514088, DCG 4.882905136959619, Average Rank 12.104119601328904, ERR 0.118756082303549
Average Validation Group Exposure disparity: 0.07129447810688502, Group Asymmetric disparity: 0.001762153525947845

Averages of last

Epoch 0, Average Validation: NDCG: 0.829508265688379, DCG 4.893289362030756, Average Rank 11.96970099667774, ERR 0.11892807457432866
Average Validation Group Exposure disparity: 0.07279578394523964, Group Asymmetric disparity: 0.0016919697341524786

Averages of last 1000 rewards: 0.8345672368199182, ndcgs: 0.8345672368199182, dcgs: 4.965944261689132
Evaluating on validation set: iteration 250/500 of epoch 0
Epoch 0, Average Validation: NDCG: 0.8300824460578085, DCG 4.897252279144651, Average Rank 11.98890365448505, ERR 0.11936064943330894
Average Validation Group Exposure disparity: 0.06649049430354217, Group Asymmetric disparity: 0.0021629870308594707
Training....
Evaluating on validation set: iteration 0/500 of epoch 1
Epoch 1, Average Validation: NDCG: 0.8277063677207471, DCG 4.883764751594846, Average Rank 11.997076411960133, ERR 0.11821186715300168
Average Validation Group Exposure disparity: 0.07557244779061299, Group Asymmetric disparity: 0.0010161198983519393

Averages of last 

In [18]:
plt_data_pl

array([[9.24320970e-01, 1.68461254e-02],
       [9.15759897e-01, 1.20337686e-02],
       [9.17154255e-01, 1.00885148e-02],
       [9.08808678e-01, 7.91118704e-03],
       [8.41275733e-01, 1.49546294e-04],
       [8.34846304e-01, 1.93747102e-03],
       [8.30971883e-01, 1.43307212e-03],
       [8.28300485e-01, 1.76084698e-03],
       [8.27174907e-01, 9.72622991e-04],
       [8.24990027e-01, 6.69106399e-04],
       [8.28755119e-01, 2.33050280e-03]])

In [ ]:
from itertools import permutations 
from scipy.optimize import linprog

In [ ]:


def assign_groups(groups):
    G = [[], []]
    for i in range(len(groups)):
        G[groups[i]].append(i)
    return G


def fair_rank(relevances, groups ,lmda = 1):
    n = len(relevances)
    pos_bias = vvector(n)
    G = assign_groups(groups)
    n_g, n_i = 0, 0
    n_g += (len(G)-1)*len(G)
    n_c = n**2 + n_g 
    
    
    c = np.ones(n_c)
    c[:n**2] *= -1
    c[n**2:] *= lmda
    A_eq = []
    #For each Row
    for i in range(n):
        A_temp = np.zeros(n_c)
        A_temp[i*n:(i+1)*n] = 1
        assert(sum(A_temp)==n)
        A_eq.append(A_temp)
        c[i*n:(i+1)*n] *= relevances[i]

    #For each coloumn
    for i in range(n):
        A_temp = np.zeros(n_c)
        A_temp[i:n**2:n] = 1
        assert(sum(A_temp)==n)
        A_eq.append(A_temp)
        #Optimization 
        c[i:n**2:n] *= pos_bias[i] 
    b_eq = np.ones(n*2) 
    A_eq = np.asarray(A_eq)
    bounds = [(0,1) for _ in range(n**2)] + [(0,None) for _ in range(n_g+n_i)]
    
    
    A_ub = []
    b_ub = np.zeros(n_g)
    sum_rels = []
    for group in G:
        #Avoid devision by zero
        sum_rel = np.max([np.sum(np.asarray(relevances)[group]), 0.01])
        sum_rels.append(sum_rel)
    comparisons = list(permutations(np.arange(len(G)),2))
    j = 0
    for a,b in comparisons:
        f = np.zeros(n_c)
        if len(G[a]) > 0 and len(G[b])>0 and sum_rels[a]/len(G[a]) >= sum_rels[b]/len(G[b]):
            for i in range(n):
                tmp1 = len(G[a]) / sum_rels[a] if i in G[a] else 0 
                tmp2 = len(G[b]) / sum_rels[b] if i in G[b] else 0 
                #f[i*n:(i+1)*n] *= max(0, sign*(tmp1 - tmp2))
                f[i*n:(i+1)*n] =  (tmp1 - tmp2)
            for i in range(n):
                f[i:n**2:n] *= pos_bias[i]
            f[n**2+j] = -1
        j += 1
        A_ub.append(f)

    res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=bounds, 
                                  method = "interior-point")#, options=dict(tol=1e-12),)
    if res.success is False:
        print("Constraint not satisfied!!")
    probabilistic_ranking = np.reshape(res.x[:n**2],(n,n))
    return probabilistic_ranking, res, res.fun

In [ ]:
from baselines import *

In [ ]:
# Linear regression
from sklearn import linear_model
model = linear_model.LinearRegression(fit_intercept=False, normalize=False)
feats, rel = dr.data
feats = np.array([item for sublist in feats for item in sublist])
rel = np.array([item for sublist in rel for item in sublist])
model.fit(feats, rel)
# predictions on validation
feats, rel = vdr.data
se_sum = 0
length = 0
predicted_rels = []
for i, query in enumerate(feats):
    rel_pred = model.predict(query)
    predicted_rels.append(rel_pred)
    se_sum += np.sum((rel_pred - rel[i])**2)
    length += len(rel[i])
print(se_sum/ length)

In [ ]:
# # for each lambda
# now for each query, estimate all relevances
# we find a fair ranking, find it's loss w.r.t true relevances
lmbdas = np.linspace(0, 0.2, 20)

plt_data = np.zeros((len(lmbdas), 2))

for j, lmbda in enumerate(lmbdas):
    test_losses = []
    test_ndcgs = []
    
    for i in range(len(predicted_rels)):
        true_rels = rel[i]
        pred_rels = predicted_rels[i]
        groups = np.array(feats[i][:, args.group_feat_id], dtype=np.int)
        n = len(groups)
        
        P, _, _ = fair_rank(pred_rels, groups, lmbda)
        
        test_loss = get_fairness_loss(P, true_rels, vvector(n), groups)
        test_losses.append(test_loss)
        
        test_ndcg = get_ndcg(P, true_rels, vvector(n))
        test_ndcgs.append(test_ndcg)
        
    plt_data[j] = [np.mean(test_ndcgs), np.mean(test_losses)]
        
        
    print("Lambda: {}, Average Test Fairness Loss: {}, Average Test NDCG: {}".format(lmbda, 
                                                     np.mean(test_losses), np.mean(test_ndcgs)))

plt.scatter(plt_data[:, 0], plt_data[:, 1])

In [ ]:
plt.scatter(plt_data[:, 0], plt_data[:, 1])
plt.scatter(plt_data_pl[:, 0], plt_data_pl[:, 1])


In [ ]:
from zehlike import *

args.lr = [0.001]
args.lambda_reward = 1.0
plt_data_4 = []
lambdas = [0.0, 1.0, 10, 100, 1000, 10000, 100000, 1000000]
args.weight_decay = [0.0]
args.epochs = [10]
disparities_mat = np.zeros((len(lambdas), 1))
ndcg_mat = np.zeros((len(lambdas), 1))
for i, lg in enumerate(lambdas):
    args.lambda_group_fairness = lg
    model = LinearModel(D=args.input_dim)
    model = demographic_parity_train(model, dr, vdr, vvector(200), args)
    results = evaluate_model(
            model,
            vdr,
            fairness_evaluation=False,
            group_fairness_evaluation=True,
            deterministic=True,
            args=args,
            num_sample_per_query=100)
    plt_data_4.append([results["ndcg"], results["avg_group_asym_disparity"]])
    ndcg_mat[i, 0], disparities_mat[i,0] = results["ndcg"], results["avg_group_asym_disparity"]

In [ ]:
plt_data_4 = np.array(plt_data_4)

In [ ]:
plt_data_4

In [ ]:
plt.scatter(plt_data_4[:, 0], plt_data_4[:, 1])


In [ ]:
def ndcg_vs_disparity_plot(plt_data_mats, names, join=False, ranges=None):
    plt.figure(figsize=(6.5, 4))
    if ranges:
        plt.xlim(ranges[0])
        plt.ylim(ranges[1])
    initialize_params()
    for i, plt_data_mat in enumerate(plt_data_mats):
        if not join:
            plt.scatter(
                plt_data_mats[i][:, 0],
                plt_data_mats[i][:, 1],
                marker="*",
                label=names[i])
        else:
            plt.plot(
                plt_data_mats[i][:, 0],
                plt_data_mats[i][:, 1],
                marker="*",
                linestyle='--',
                label=names[i])
    plt.legend(fontsize=14)
#     plt.title("Utility-Fairness trade-off",y=-0.30)
    plt.xlabel("NDCG")
    plt.ylabel(r'$\hat{\mathcal{D}}_{\rm group}$', fontsize=16)
    plt.grid()
    plt.savefig('./plots/german_tradeoff.pdf', bbox_inches='tight')
    plt.show()

In [ ]:
ndcg_vs_disparity_plot([plt_data, plt_data_pl,  plt_data_4], ["Post-Processing ($\lambda \in [0, 0.2]$)",
                      "Our Method ($\lambda \in [0,100]$ )", 
                      "Zehlike \& Castillo 2018 ($\lambda \in [0, 10^6]$)"], join=True, ranges=[[0.60, 0.85], [0.00, 0.054]])

In [ ]:
ndcg_vs_disparity_plot([plt_data, plt_data_pl], ["Post-Processing ($\lambda \in [0, 0.2]$)",
                      "Our Method ($\lambda \in [0,100]$ )"], join=True, ranges=[[0.60, 0.85], [0.00, 0.054]])